In [4]:
from elasticsearch import Elasticsearch
import pandas as pd

In [10]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic","pass"),
    ca_certs="certifcate path")
es.ping()

True

## Data Preparation

In [13]:
df = pd.read_csv('myntra_products_catalog.csv').loc[:499]
df.fillna("None",inplace=True)
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


## Convert relelvent field to vector using bert

In [9]:
import sklearn
import torch

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
df["DescriptionVector"]= df["Description"].apply(lambda x: model.encode(x))

#### Create new index is ES

In [ ]:
from indexMapping import indexMapping
es.indices.create(index="all_products",mappings=indexMapping)

#### Ubgest data into index
#### ELasticsearch is a json DB , convert df to json


In [ ]:
record_list=df.to_dict("records")
for record in record_list:
    try:
        es.index(index="all_products", document=record, id=record["ProductID"])
    except Exception as e:
        print(e)
es.count(index="all_products")
# ID Is not auto generated by ES , we chose it

### Search the data 

In [ ]:
input_keyword = "Blue Shoes"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "DescriptionVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 2,
    "num_candidates" : 500, 
}

res = es.knn_search(index="all_products", knn=query , source=["ProductName","Description"])
res["hits"]["hits"]